# [Module 4.0] Make Realtime Endpoint and Inference

**이전의 배치성 추론과 다르게 한 건씩 실시간으로 추론을 해보겠습니다.**

이 노트북은 아래와 같은 작업을 합니다.
- Endpoint Config 생성
- RealTime Endpoint 생성
- Predictor 생성
- 추론을 위한 테스트 데이터 포맷 작성
- 추론 실행 (20명)

소요시간은 약 10분 소요 됩니다.

In [5]:
import boto3

import pandas as pd
import sagemaker

from time import strftime, gmtime

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [6]:
%store -r 

## EndpointConfig 생성
- Realtime Endpoint를 만들기 위한 설정 파일을 만듦니다.
    - model_name은 기존에 만든 model을 사용합니다. 위의 %store -r을 통해서 해당 이름을 가져 옵니다.

In [9]:
client = boto3.client("sagemaker")
endpoint_config_name = "churn-inference-pipeline-" + timestamp_prefix
response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants = [
        {
            'VariantName' : "AllTraffic",
            'ModelName' : model_name,
            'InitialInstanceCount' : 1,
            'InstanceType' : 'ml.m4.xlarge'
#             'InstanceType' : 'local'            
        }
    ]
)

ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-east-2:057716757052:endpoint-config/churn-inference-pipeline-2020-08-09-01-26-51".

## Endpoint 생성

In [10]:
churn_endpoint_name = 'churn-inference-pipeline-endpoint-' + timestamp_prefix
churn_endpoint = client.create_endpoint(
    EndpointName = churn_endpoint_name,
    EndpointConfigName = endpoint_config_name
)

#### **EndpointStatus 가 InService**가 될때까지 기다려 주세요. 그렇지 않으면 아래 코드가 에러나 납니다.
EndpointStatus 가 Pending --> Creating --> In Service로 변경 됩니다.

#### 아래는 약 10분 정도가 소요 됩니다. 
아래 셀을 주기적으로 키보드로 Shift-Enter 를 눌러서 실행해 주세요

In [12]:
client.describe_endpoint(EndpointName = churn_endpoint_name)

{'EndpointName': 'churn-inference-pipeline-endpoint-2020-08-09-01-26-51',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:057716757052:endpoint/churn-inference-pipeline-endpoint-2020-08-09-01-26-51',
 'EndpointConfigName': 'churn-inference-pipeline-2020-08-09-01-26-51',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn@sha256:1476311657fe444be8f95f472a8a91a3da577d558e1046678ee76c611f7132ca',
     'ResolutionTime': datetime.datetime(2020, 8, 9, 1, 56, 23, 65000, tzinfo=tzlocal())},
    {'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:b0f7d76963f88f9890bfa3288e256d4cc14308ace32a036f77b977d8cf4319a5',
     'ResolutionTime': datetime.

### Create Predictor
- 위에서 생성한 Endpoint에 CSV 형태의 입력 데이타를 받을 수 있게 csv_serializer를 지정 함

In [13]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
import sagemaker
sagemaker_session = sagemaker.Session()

predictor = RealTimePredictor(
    endpoint = churn_endpoint_name,
    sagemaker_session = sagemaker_session,
    serializer = csv_serializer,
    content_type = CONTENT_TYPE_CSV,
    accept = CONTENT_TYPE_JSON
)


### 추론 위한 입력 포맷을 만듦
- 이런 형태로 입력이 됨 (에:)
    - ' KS,186,510,400-6454,no,no,0,137.8,97,23.43,187.7,118,15.95,146.4,85,6.59,8.7,6,2.35,1' 

In [14]:

def make_inference_format(sample):
    instance = str()
    for i, token in enumerate(sample):
        # print(token)
        if i > 0:
            instance = instance  + ',' + str(token) 
        else:
            instance = instance  +  str(token) 
    return instance



### Test 데이타에서 20명을 추론
- 20명 중에서 7번째 고객이 이탈이 가능하다고 결과를 냄. 

In [17]:
import os
os.getcwd()

'/home/ec2-user/SageMaker/churn-prediction-workshop/working'

In [18]:
test_df = pd.read_csv("../churn_data/batch_transform_test.csv", header=None)

for i in range(20):
    sample = test_df.iloc[i,:]
    instance = make_inference_format(sample)
    print("instance: \n", instance)

    payload = instance
    churn_result = predictor.predict(payload)
    print("Churn result?: \n", churn_result)
    print("")

instance: 
 KS,186,510,400-6454,no,no,0,137.8,97,23.43,187.7,118,15.95,146.4,85,6.59,8.7,6,2.35,1


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from container-1 with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/churn-inference-pipeline-endpoint-2020-08-09-01-26-51 in account 057716757052 for more information.

## Delete Endpoint
- 작업을 완료 했으면, 필요 없는 리소스는 삭제

In [12]:
client.delete_endpoint(EndpointName=churn_endpoint_name)

{'ResponseMetadata': {'RequestId': 'aa72b69b-cf69-4ffa-aa56-bb3a7ede5f49',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'aa72b69b-cf69-4ffa-aa56-bb3a7ede5f49',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 03 Aug 2020 10:07:01 GMT'},
  'RetryAttempts': 0}}